In [1]:
import psycopg2
from faker import Faker
import random
import csv


fake = Faker()

In [2]:
def get_connection():
    return psycopg2.connect(
        host="localhost",
        database="5310 project",
        user="postgres",
        password="123"
    )


In [3]:
def generate_shippings(n=5000):
    conn = get_connection()
    cursor = conn.cursor()
    
    # Fetch all unique order_ids from the customer_order table
    cursor.execute("SELECT DISTINCT order_id FROM customer_order")
    orders = [order[0] for order in cursor.fetchall()]  # Flatten the list of tuples

    # Check if there are enough orders
    if len(orders) < n:
        print(f"Error: Not enough unique order_ids. Need at least {n} unique order_ids.")
        cursor.close()
        conn.close()
        return

    # Shuffle the list of orders to ensure randomness
    random.shuffle(orders)

    shipping_companies = ["DHL", "FedEx", "UPS", "USPS", "DoorDash", "ShipBob", "Postmates", "Instacart"]

    shippings = []

    for i in range(n):
        order_id = orders[i]
        shipping_company = random.choice(shipping_companies)
        delivery_time = random.randint(1, 14)  # Delivery time in days
        
        # Save the shipping info to a list
        shippings.append((order_id, shipping_company, delivery_time))

        # Insert the shipping info into the database
        cursor.execute(
            "INSERT INTO shippings (order_id, shipping_company, delivery_time) VALUES (%s, %s, %s)",
            (order_id, shipping_company, delivery_time)
        )

    conn.commit()
    cursor.close()
    conn.close()
    
    # Write the generated ratings data to a CSV file
    with open('C:/Users/Yihua/Desktop/5310 project/shippings.csv', 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['order_id', 'shipping_company', 'delivery_time'])
        csv_writer.writerows(shippings)

if __name__ == "__main__":
    generate_shippings()